# 1. Importing Libraries

In [21]:
from pandas import Series, DataFrame
import pandas as pd
import numpy as np
#import plotly.plotly as py
#py.sign_in('suryak1210', 'nd72ait1tg')
import folium 
from folium.element import IFrame
from folium.plugins import MarkerCluster
from folium import plugins
import json

# 2. Data Cleaning and Data Enrichment 

#### Warning
This part of the code takes a long time as we are reading the random sample of the data from huge file containing approx 70 million rows.
We have run this code once and we are saving dataset of interest to the drice and reading that file for our analysis.

####   2.1 ****Function to extract Zipcodes from coordinates.****

In [52]:
def cordnts_zip(a,b):
    try:
        location = search.by_coordinate(a,b)
        lctn_zip = location[0]["Zipcode"]
    except:
        lctn_zip = ''
    return lctn_zip

#### 2.2 Set seed and parameters for randomsampling and Initialize empty dataset.

In [10]:
np.random.seed(456)
nlinesrandomsample = 100000
nlines = list([13158262,12324935,11562783,11130304,11225063,12315488,11312676]) #Number of lines in each of the raw datasets
#files = os.listdir('C:/MyGit/MSBA/nyc_taxi/Monthly Raw Datasets')
nyc_taxi_may_nov_2015 = pd.DataFrame()

#### 2.3 Load the datasets from each file and append onto one file.

In [17]:
for i in range(8):
    lines2skip = np.random.choice(np.arange(1,nlines[i]+1), \
    (nlines[i]-nlinesrandomsample), replace=False)
    nyc_taxi_temp = pd.read_csv('C:/MyGit/MSBA/nyc_taxi/Monthly Raw Datasets/'+files[i],skiprows = lines2skip)
    nyc_taxi_temp = nyc_taxi_temp.rename(columns = {'RateCodeID':'RatecodeID'})
    nyc_taxi_may_nov_2015 = nyc_taxi_may_nov_2015.append(nyc_taxi_temp) 

#### 2.4 Remove temporary variables

In [15]:
del nyc_taxi_temp,nlines,lines2skip,i,files,nlinesrandomsample

#### 2.5 Convert datetime object column to datetime series

In [ ]:
nyc_taxi_may_nov_2015['tpep_pickup_datetime'] = pd.to_datetime(\
                nyc_taxi_may_nov_2015['tpep_pickup_datetime'],dayfirst = True)
nyc_taxi_may_nov_2015['tpep_dropoff_datetime'] = pd.to_datetime(\
                nyc_taxi_may_nov_2015['tpep_dropoff_datetime'],dayfirst = True)
nyc_taxi_may_nov_2015['Time'] = pd.to_datetime(nyc_taxi_may_nov_2015['Time'])

In [ ]:
nyc_taxi_may_nov_2015['Month'] = nyc_taxi_may_nov_2015['tpep_pickup_datetime'].dt.month
nyc_taxi_may_nov_2015['Time'] = nyc_taxi_may_nov_2015['tpep_pickup_datetime'].dt.time
nyc_taxi_may_nov_2015['Day'] = nyc_taxi_may_nov_2015['tpep_pickup_datetime'].dt.day

#### 2.6 Remove erroneous coordinates (0,0) from the dataset

In [ ]:
nyc_taxi_may_nov_2015 = \
nyc_taxi_may_nov_2015[(nyc_taxi_may_nov_2015['pickup_latitude'] != 0 ) & \
(nyc_taxi_may_nov_2015['pickup_longitude']!=0 )]

#### 2.7 Set index for the dataset to be unique

In [ ]:
nyc_taxi_may_nov_2015 = nyc_taxi_may_nov_2015.set_index(np.arange(0,788757))

####  2.8 Add columns for Pick-up and drop-off zipcodes

In [ ]:
nyc_taxi_may_nov_2015['pickup_zip'] = nyc_taxi_may_nov_2015.apply(lambda x:\
cordnts_zip(x['pickup_latitude'],x['pickup_longitude']),axis=1)   

nyc_taxi_may_nov_2015['dropoff_zip'] = nyc_taxi_may_nov_2015.apply(lambda x:\
cordnts_zip(x['dropoff_latitude'],x['dropoff_longitude']),axis=1)  

#### 2.9 Saving the cleaned and enriched file

In [ ]:
nyc_taxi_may_nov_2015.to_csv('nyc_taxi_may_nov_2015.csv')

#### 2.10 Set seed and parameters for randomsampling and Initialize empty dataset. 

In [ ]:
np.random.seed(456)
nlinesrandomsample = 100000
nlines = list([961986,941219,1085676,1179044,1289699,1212277,987245,804125])
files = os.listdir('Citibike Data')
citi_bike_may_dec_2015 = pd.DataFrame()

#### 2.11 Load the datasets from each file and append onto one file.

In [ ]:
for i in range(8):
    lines2skip = np.random.choice(np.arange(1,nlines[i]+1), \
    (nlines[i]-nlinesrandomsample), replace=False)
    bike_temp = pd.read_csv('Citibike Data/'+files[i],skiprows = lines2skip)
    citi_bike_may_dec_2015 = citi_bike_may_dec_2015.append(bike_temp)

#### 2.12 Remove temporary variables

In [ ]:
del nyc_taxi_temp,nlines,lines2skip,i,files,nlinesrandomsample

#### 2.13 Convert datetime object column to datetime series

In [ ]:
citi_bike_may_dec_2015['starttime'] = pd.to_datetime(\
                                            citi_bike_may_dec_2015['starttime'])
citi_bike_may_dec_2015['stoptime'] = pd.to_datetime(\
                                            citi_bike_may_dec_2015['stoptime'])
                                            
citi_bike_may_dec_2015['month'] = pd.DatetimeIndex(citi_bike_may_dec_2015
                                                 ['starttime']).month
citi_bike_may_dec_2015['startdate'] = pd.DatetimeIndex(citi_bike_may_dec_2015
                                                 ['starttime']).date

#### 2.14 Remove erroneous coordinates (0,0) from the dataset

In [ ]:
citi_bike_may_dec_2015 = \
citi_bike_may_dec_2015[(citi_bike_may_dec_2015['start station latitude'] != 0 ) & \
(citi_bike_may_dec_2015['start station longitude']!=0 )]  

citi_bike_may_dec_2015 = \
citi_bike_may_dec_2015[(citi_bike_may_dec_2015['end station latitude'] != 0 ) & \
(citi_bike_may_dec_2015['end station longitude']!=0 )]  


####  2.15 Add columns for Pick-up and drop-off zipcodes

In [ ]:
citi_bike_may_dec_2015['pickup_zip'] = citi_bike_may_dec_2015.apply(lambda x:\
cordnts_zip(x['start station latitude'],x['start station longitude']),axis=1)   

citi_bike_may_dec_2015['dropoff_zip'] = citi_bike_may_dec_2015.apply(lambda x:\
cordnts_zip(x['end station latitude'],x['end station longitude']),axis=1)

#### 2.16 Dropping data for december for uniformity with NYC_taxi data

In [ ]:
citi_bike_may_nov_2015 = citi_bike_may_dec_2015[citi_bike_may_dec_2015['month']!= 12]

#### 2.17 Saving the cleaned and enriched file

In [ ]:
citi_bike_may_nov_2015.to_csv('citi_bike_may_nov_2015.csv')

# 3. Plotting of the NYC Taxi and NYC Bike data as heat maps

In [99]:
cd 'E:\\documents-export-2016-08-14'


E:\documents-export-2016-08-14


In [100]:
nyc_taxi_may_nov_2015 =  pd.read_csv('nyc_taxi_may_nov_2015.csv')

In [26]:
bike_may_nov_2015 = pd.read_csv('citi_bike_may_nov_2015.csv')

## 3.1 Plotting a heat map NYC taxi across zip

#### 3.1.1 Reading a geo_json file with zip codes of New York

In [101]:
json_data=open('zip.geojson','r').read()
data = json.loads(json_data)
zip_list = list()
for i in range(0,len(data['features'])):
    zip_list.append(data['features'][i]['properties']['postalCode'])

zip_list= DataFrame(zip_list)
zip_list.columns = ['zip']
zip_list['zip'] = zip_list['zip'].astype(str)

#### 3.1.2 Counting the number of taxis in each zip for pickup and dropoff

In [102]:
nyc_taxi_pickup= nyc_taxi_may_nov_2015['pickup_zip'].value_counts()   
nyc_taxi_pickup.index = (nyc_taxi_pickup.index).astype(int).astype(str)
nyc_taxi_pickup = nyc_taxi_pickup.reset_index()
nyc_taxi_pickup.columns = ['zip','count']

#### 3.1.3 Creating a new dataframe for plotting the data on the map.

In [103]:
nyc_taxi_pickup = pd.merge(nyc_taxi_pickup,zip_list,how = 'right')
nyc_taxi_pickup['count'] = nyc_taxi_pickup['count'].fillna(0.0).astype(int)

### 3.1.4 Plotting heat maps for the pickup zip for NYC taxi 

##### Reading the geojson file with the zip code of New York

In [35]:
geo_path = r'zip.geojson'
heatmap_scale = list()
threshold = [10,20,50,70,85,100]
for i in threshold :
    heatmap_scale.append(int(nyc_taxi_pickup['count'].max() * (i/100.0)))

map_nyc_taxi_pickup = folium.Map(location=[40.773124694800003, -73.952171325699993], zoom_start=10)
map_nyc_taxi_pickup.geo_json(geo_path=geo_path, data=nyc_taxi_pickup, \
                data_out = 'nyc_zip_test.json',
             columns=['zip', 'count'],
             threshold_scale= heatmap_scale,
             key_on='feature.properties.postalCode',
             fill_color='YlOrRd', fill_opacity=0.9, line_opacity=0.9,
             legend_name='Number of Pickups')
map_nyc_taxi_pickup.create_map(path='map_nyc_taxi_pickup.html')

C:\Users\SuryaNarayana\Anaconda2\lib\site-packages\ipykernel\__main__.py:14: FutureWarning: Map.create_map is deprecated. Use Map.save instead


##### Plotting the map

In [36]:
map_nyc_taxi_pickup

#### 3.1.5 Counting the number of taxis in each zip for pickup and dropoff

In [37]:
nyc_taxi_dropoff= nyc_taxi_may_nov_2015['dropoff_zip'].value_counts()   
nyc_taxi_dropoff.index = (nyc_taxi_dropoff.index).astype(int).astype(str)
nyc_taxi_dropoff = nyc_taxi_dropoff.reset_index()
nyc_taxi_dropoff.columns = ['zip','count']

#### 3.1.6 Creating a new dataframe for plotting the data on the map.

In [38]:
nyc_taxi_dropoff = pd.merge(nyc_taxi_dropoff,zip_list,how = 'right')
nyc_taxi_dropoff['count'] = nyc_taxi_dropoff['count'].fillna(0.0).astype(int)

### 3.1.7 Plotting heat maps for the dropoff zip for NYC taxi 

##### Reading the geojson file with the zip code of New York

In [39]:
geo_path = r'zip.geojson'
heatmap_scale = list()
threshold = [10,20,50,70,85,100]
for i in threshold :
    heatmap_scale.append(int(nyc_taxi_pickup['count'].max() * (i/100.0)))

map_nyc_taxi_dropoff = folium.Map(location=[40.773124694800003, -73.952171325699993], zoom_start=10)
map_nyc_taxi_dropoff.geo_json(geo_path=geo_path, data=nyc_taxi_dropoff, \
                data_out = 'nyc_zip_test.json',
             columns=['zip', 'count'],
             threshold_scale= heatmap_scale,
             key_on='feature.properties.postalCode',
             fill_color='YlOrRd', fill_opacity=0.9, line_opacity=0.9,
             legend_name='Number of Pickups')
map_nyc_taxi_dropoff.create_map(path='map_nyc_taxi_dropoff.html')

C:\Users\SuryaNarayana\Anaconda2\lib\site-packages\ipykernel\__main__.py:14: FutureWarning: Map.create_map is deprecated. Use Map.save instead


##### Plotting the map

In [40]:
map_nyc_taxi_dropoff

## 3.2 Plotting a heat map NYC citi bike across zip

#### 3.2.1 Counting the number of taxis in each zip for pickup and dropoff

In [41]:
bike_pickup = bike_may_nov_2015['pickup_zip'].value_counts()
bike_pickup.index =(bike_pickup.index).astype(str)
bike_pickup = bike_pickup.reset_index()
bike_pickup.columns = ['zip','count']

#### 3.2.3 Creating a new dataframe for plotting the data on the map.

In [42]:
bike_pickup = pd.merge(bike_pickup,zip_list,how = 'right')
bike_pickup['count'] = bike_pickup['count'].fillna(0.0).astype(int)

### 3.2.4 Plotting heat maps for the pickup zip for NYC taxi 

##### Reading the geojson file with the zip code of New York

In [43]:
geo_path = r'zip.geojson'
heatmap_scale = list()
threshold = [10,20,50,70,85,100]
for i in threshold :
    heatmap_scale.append(int(bike_pickup['count'].max() * (i/100.0)))

map_bike_pickup = folium.Map(location=[40.773124694800003, -73.952171325699993], zoom_start=10)
map_bike_pickup.geo_json(geo_path=geo_path, data=bike_pickup, \
                data_out = 'nyc_zip_test.json',
             columns=['zip', 'count'],
             threshold_scale= heatmap_scale,
             key_on='feature.properties.postalCode',
             fill_color='YlOrRd', fill_opacity=0.9, line_opacity=0.9,
             legend_name='Number of Searches')
map_bike_pickup.create_map(path='map_bike_pickup.html')  

C:\Users\SuryaNarayana\Anaconda2\lib\site-packages\ipykernel\__main__.py:14: FutureWarning: Map.create_map is deprecated. Use Map.save instead


##### Plotting the map

In [45]:
map_bike_pickup

#### 3.2.5 Counting the number of taxis in each zip for pickup and dropoff

In [46]:
bike_dropoff = bike_may_nov_2015['dropoff_zip'].value_counts()
bike_dropoff.index =(bike_dropoff.index).astype(str)
bike_dropoff = bike_dropoff.reset_index()
bike_dropoff.columns = ['zip','count']

#### 3.2.6 Creating a new dataframe for plotting the data on the map.

In [47]:
bike_dropoff = pd.merge(bike_dropoff,zip_list,how = 'right')
bike_dropoff['count'] = bike_dropoff['count'].fillna(0.0).astype(int)

### 3.2.7 Plotting heat maps for the pickup zip for NYC taxi 

##### Reading the geojson file with the zip code of New York

In [48]:
geo_path = r'zip.geojson'
heatmap_scale = list()
threshold = [10,20,50,70,85,100]
for i in threshold :
    heatmap_scale.append(int(bike_pickup['count'].max() * (i/100.0)))

map_bike_dropoff = folium.Map(location=[40.773124694800003, -73.952171325699993], zoom_start=10)
map_bike_dropoff.geo_json(geo_path=geo_path, data=bike_dropoff, \
                data_out = 'nyc_zip_test.json',
             columns=['zip', 'count'],
             threshold_scale= heatmap_scale,
             key_on='feature.properties.postalCode',
             fill_color='YlOrRd', fill_opacity=0.9, line_opacity=0.9,
             legend_name='Number of Searches')
map_bike_dropoff.create_map(path='map_bike_dropoff.html')  

C:\Users\SuryaNarayana\Anaconda2\lib\site-packages\ipykernel\__main__.py:14: FutureWarning: Map.create_map is deprecated. Use Map.save instead


##### Plotting the map

In [50]:
map_bike_dropoff

# 4 Drill down on Zips for NYC taxi and Bikes

#### 4.1 Setting the NYC co-ordinates for the Map.

In [31]:
NY_COORDINATES = (40.773124694800003, -73.952171325699993)